ΑΡΙΣΤΟΤΕΛΕΙΟ ΠΑΝΕΠΙΣΤΗΜΙΟ ΘΕΣΣΑΛΟΝΙΚΗΣ

ΠΟΛΥΤΕΧΝΙΚΗ ΣΧΟΛΗ - ΤΜΗΜΑ ΗΛΕΚΤΡΟΛΟΓΩΝ ΜΗΧΑΝΙΚΩΝ ΚΑΙ ΜΗΧΑΝΙΚΩΝ ΥΠΟΛΟΓΙΣΤΩΝ

ΤΟΜΕΑΣ ΗΛΕΚΤΡΟΝΙΚΗΣ ΚΑΙ ΥΠΟΛΟΓΙΣΤΩΝ


# ΕΠΙΧΕΙΡΗΣΙΑΚΗ ΕΡΕΥΝΑ

## ΕΡΓΑΣΙΑ / PROJECT 2024

**Ονοματεπώνυμο**: Χριστοφορίδης Χρήστος

**ΑΕΜ**: 10395

**Email**: christoscs@ece.auth.gr

In [215]:
## IMPORTS ##
import gurobipy as gp
from gurobipy import GRB
import numpy as np

## ΘΕΜΑ 1 - Airport Landing Schedule

Έστω $A$ το σύνολο των αεροσκαφών δηλαδή $Α = \{1,2,3,...,10\}$

- ${earliest\_arrival}_i$ : Νωρίτερος χρόνος άφιξης του αεροσκάφους $i$.

- ${estimated\_arrival}_i$ : Εκτιμώμενος χρόνος άφιξης του αεροσκάφους $i$.

- ${latest\_arrival}_i$ : Αργότερος χρόνος άφιξης του αεροσκάφους $i$.

- ${min\_time\_between\_landings}_{ij}$ : Ελάχιστο χρονικό διάστημα μεταξύ των προσγειώσεων των αεροσκαφών $i$ και $j$

- ${early\_penalty}_i$: Πρόστιμο προωρότητας του αεροσκάφους $i$
- ${late\_penalty}_i$: Πρόστιμο καθυστέρησης του αεροσκάφους $i$

Ορίζω νέες μεταβλητές:
- ${landing\_time}_i$ : Αντιπροσωπεύει την χρονική στιγμή που φτάνει το αεροσκάφος $i$.
- $δ_{ij}$ : Ισούται με $1$ όταν η πτήση $i$ φτάνει πριν την πτήση $j$, αλλιώς ισούται με $0$.
- ${z\_p}_i$: Χρόνος καθυστέρησης, ισούται με max$[0,{landing\_time}_i - {estimated\_arrival}_i]$
- ${z\_n}_i$: Χρόνος προωρότητας, ισούται με max$[0,{estimated\_arrival}_i - {landing\_time}_i]$
- $M$: Ένας μεγάλος αριθμός.

Προκύπτουν λοιπόν οι παρακάτω περιορισμοί:

1. Προφανώς ένα από τα δύο αεροσκάφη προσγειώνεται πριν από το άλλο άρα ένα ισούται με μονάδα και το άλλο με μηδέν:
- $δ_{ij} + δ_{ji} = 1, i \ne j$
2. Η ώρα άφιξης πρέπει να είναι μεγαλύτερη της ελάχιστης και μικρότερη της μέγιστης:
- $ {landing\_time}_i \ge {earliest\_arrival}_i$
- $ {landing\_time}_i \le {latest\_arrival}_i$
3. Αν ο χρόνος άφιξης είναι μεγαλύτερος από τον εκτιμώμενο χρόνο τότε το ${z\_p}_i$ είναι θετικό, ενώ αν αν είναι μικρότερος τότε το ${z\_n}_i$ είναι θετικό:
- ${z\_p}_i \ge {landing\_time}_i - {estimated\_arrival}_i $
- ${z\_n}_i \ge {estimated\_arrival}_i - {landing\_time}_i $
4. Αυτός ο περιορισμός διασφαλίζει το ελάχιστο χρονικό διάστημα μεταξύ των προσγειώσεων. Αν το αεροσκάφος $j$ προσγειωθεί πριν το $i$, το $δ_{ji}$ είναι ίσο με $1$ και άρα ο περιορισμός δεν είναι δεσμευτικός, λόγω του μεγάλου αριθμού Μ:
- ${landing\_time}_j - {landing\_time}_i \ge {min\_time\_between\_landings}_{ij} - δ_{ji}M$


Η συνάρτηση συνολικού προστίμου που θέλω και να ελαχιστοποιήσω είναι η παρακάτω:
$$total\_penalty = \sum_{i=1}^{10} ({late\_penalty}_i{z\_p}_i + {early\_penalty}_i{z\_n}_i) $$

Ο κώδικας που ακολουθεί μοντελοποιεί το παραπάνω πρόβλημα όπως περιγράφηκε και βρίσκει την βέλτιστη λύση.

In [216]:
## DATA ##
aircrafts = 10
earliest_arrival = [129, 195, 89, 96, 110, 120, 124, 126, 135, 160]
estimated_arrival = [155, 258, 96, 106, 123, 135, 138, 140, 150, 180]
latest_arrival = [559, 744, 510, 521, 555, 576, 577, 573, 591, 657]
early_penalty = [10, 10, 30, 30, 30, 30, 30, 30, 30, 30]
late_penalty = [10, 10, 30, 30, 30, 30, 30, 30, 30, 30]
min_time_between_landings = [
    [0, 3, 15, 15, 15, 15, 15, 15, 15, 15],
    [3, 0, 15, 15, 15, 15, 15, 15, 15, 15],
    [15, 15, 0, 8, 8, 8, 8, 8, 8, 8],
    [15, 15, 8, 0, 8, 8, 8, 8, 8, 8],
    [15, 15, 8, 8, 0, 8, 8, 8, 8, 8],
    [15, 15, 8, 8, 8, 0, 8, 8, 8, 8],
    [15, 15, 8, 8, 8, 8, 0, 8, 8, 8],
    [15, 15, 8, 8, 8, 8, 8, 0, 8, 8],
    [15, 15, 8, 8, 8, 8, 8, 8, 0, 8],
    [15, 15, 8, 8, 8, 8, 8, 8, 8, 0],
]
M = max(latest_arrival) - min(earliest_arrival) # Big Number

In [217]:
## MODEL ##
model = gp.Model("Aircraft_Landing_Schedule")

# variables
landing_time = model.addVars(aircrafts, vtype=GRB.CONTINUOUS,name="landing_time")
z_p = model.addVars(aircrafts, vtype=GRB.CONTINUOUS,name="z_p")
z_n = model.addVars(aircrafts, vtype=GRB.CONTINUOUS,name="z_n")
delta = model.addVars(aircrafts, aircrafts, vtype=GRB.BINARY,name="delta")

# constraints
for i in range(aircrafts):
    model.addConstr(landing_time[i] >= earliest_arrival[i])
    model.addConstr(landing_time[i] <= latest_arrival[i])
    model.addConstr(z_p[i] >= landing_time[i]-estimated_arrival[i])
    model.addConstr(z_n[i] >= estimated_arrival[i]-landing_time[i])
for i in range(aircrafts):
    for j in range(aircrafts):
        if i != j:
            model.addConstr(delta[i,j] + delta[j,i] == 1)
            model.addConstr(landing_time[j]-landing_time[i] >= min_time_between_landings[i][j]-delta[j,i]*M)

# cost
total_penalty = gp.quicksum(early_penalty[i]*z_n[i] + late_penalty[i]*z_p[i] for i in range(aircrafts)) 

# optimize
model.setObjective(total_penalty, GRB.MINIMIZE)
model.optimize()

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i5-6400 CPU @ 2.70GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 220 rows, 130 columns and 510 nonzeros
Model fingerprint: 0xb1a23624
Variable types: 30 continuous, 100 integer (100 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+02]
  Objective range  [1e+01, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+02]
Found heuristic solution: objective 13850.000000
Presolve removed 120 rows and 55 columns
Presolve time: 0.00s
Presolved: 100 rows, 75 columns, 300 nonzeros
Variable types: 30 continuous, 45 integer (45 binary)

Root relaxation: objective 0.000000e+00, 41 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

In [218]:
## RESULTS ##
copy = []
if model.status == GRB.OPTIMAL:
    print("Βέλτιστη Λύση:")
    for i in range(aircrafts):
        copy.append(landing_time[i].x)
        print("Αεροσκάφος ", i+1, "- Χρόνος προσγείωσης: ", landing_time[i].x)
    order = np.argsort(copy)
    print("Τα αεροσκάφη προσγειώθηκαν με την σειρά: ",end='')
    for i in order:
        print(str(i+1)+", ",end='')
    print()
    print("Το συνολικό πρόστιμο είναι: " + str(model.ObjVal))
else:
    print("Δεν βρέθηκε βέλτιστη λύση.")

Βέλτιστη Λύση:
Αεροσκάφος  1 - Χρόνος προσγείωσης:  165.0
Αεροσκάφος  2 - Χρόνος προσγείωσης:  258.0
Αεροσκάφος  3 - Χρόνος προσγείωσης:  96.0
Αεροσκάφος  4 - Χρόνος προσγείωσης:  106.0
Αεροσκάφος  5 - Χρόνος προσγείωσης:  118.0
Αεροσκάφος  6 - Χρόνος προσγείωσης:  126.0
Αεροσκάφος  7 - Χρόνος προσγείωσης:  134.0
Αεροσκάφος  8 - Χρόνος προσγείωσης:  142.0
Αεροσκάφος  9 - Χρόνος προσγείωσης:  150.0
Αεροσκάφος  10 - Χρόνος προσγείωσης:  180.0
Τα αεροσκάφη προσγειώθηκαν με την σειρά: 3, 4, 5, 6, 7, 8, 9, 1, 10, 2, 
Το συνολικό πρόστιμο είναι: 700.0


Επομένως από τα παραπάνω προκύπτει πως ο βέλτιστος προγραμματισμός των αεροπλάνων είναι ο εξής: $3-4-5-6-7-8-9-1-10-2$

## ΘΕΜΑ 2 - Oil Delivery

Έστω ότι:
- $n$: Ο αριθμός των πόλεων (μαζί με το διυλιστήριο)
- $d_{ij}$: Η απόσταση από την πόλη $i$ στην πόλη $j$
- $D_i$: Η ζήτηση της πόλης $i$
- $C$: Η χωρητικότητα κάθε δεξαμενόπλοιου
- $v$: Ο αριθμός των οχημάτων(ή διαδρομών) που απαιτούνται $(v = \lceil \frac {\sum_{i=1}^n D_i} C \rceil)$

Ορίζω μεταβλητές:
- $x_{kij}$: Ισούται με $1$ όταν το όχημα $k$ κάνει διαδρομή από την πόλη $i$ στην πόλη $j$, αλλιώς ισούται με $0$.
- $f_{kij}$: Πόσα λίτρα μεταφέρονται από την πόλη $i$ στην πόλη $j$ από το όχημα $k$.

Οι περιορισμοί που προκύπτουν είναι οι εξής:

1. Κάθε πόλη (εκτός του διυλιστήριου) την επισκέπτομαι ακριβώς μία φορά(συμμετρικά αποχωρώ ακριβώς μία φορά) από μόνο ένα όχημα:
- $\sum_{k=1}^v\sum_{j=1}^n x_{kij} = 1~~\forall i=2,...,n$
- $\sum_{k=1}^v\sum_{i=1}^n x_{kij} = 1~~\forall j=2,...,n$

2. Περιορισμός για την ροή πετρελαίου και την ικανοποίηση ζήτησης:
- $\sum_{j=1}^n f_{kij} - \sum_{j=1}^n f_{kji} = D_i \sum_{j=1}^n x_{kij},~~ i=2,...,n~,\forall k$

3. Περιορισμός χωρητικότητας:
- $0 \le f_{kij} \le C x_{kij},~~ \forall i,j,k$

4. Κάθε όχημα ξεκινάει και τελειώνει στο διυλιστήριο:
- $\sum_{j=2}^n x_{k0j}=1,~~ \forall k$
- $\sum_{i=2}^n x_{ki0}=1,~~ \forall k$

Στόχος μου είναι να ελαχιστοποιήσω την συνολική απόσταση που διανύεται δηλαδή να ελαχιστοποιήσω την παράσταση:
$$\sum_{k=1}^v \sum_{i=1}^n \sum_{j=1}^n d_{ij} x_{kij}$$

Ο κώδικας που ακολουθεί μοντελοποιεί το παραπάνω πρόβλημα όπως περιγράφηκε και βρίσκει την βέλτιστη λύση.

In [219]:
## DATA ##
cities = ['Ω', 'Α', 'Β', 'Γ', 'Δ', 'Ε', 'ΣΤ']
D = [0, 14000, 3000, 6000, 16000, 15000, 5000]
C = 39000
d = [
    [0, 148, 55, 32, 70, 140, 73],
    [148, 0, 93, 180, 99, 12, 72],
    [55, 93, 0, 85, 20, 83, 28],
    [32, 180, 85, 0, 100, 174, 99],
    [70, 99, 20, 100, 0, 85, 49],
    [140, 12, 83, 174, 85, 0, 73],
    [73, 72, 28, 99, 49, 73, 0]
]

n = len(cities)
v = int(np.ceil(sum(D)/C))

In [220]:
## MODEL ##
model = gp.Model('Oil_Delivery')

# variables
x = model.addVars(v, n, n, vtype=GRB.BINARY, name='x')
f = model.addVars(v, n, n, vtype=GRB.CONTINUOUS, name='f')

# constraints
# 1
for i in range(1, n):
    model.addConstr(gp.quicksum(x[k, i, j] for k in range(v) for j in range(n)) == 1)
    model.addConstr(gp.quicksum(x[k, j, i] for k in range(v) for j in range(n)) == 1)
    
# 2
for k in range(v):
    for i in range(1,n):
        model.addConstr(gp.quicksum(f[k, i, j] for j in range(n)) -gp.quicksum(f[k, j, i] for j in range(n)) == D[i]*gp.quicksum(x[k, i, j] for j in range(n)))

# 3
for k in range(v):
    for i in range(n):
        for j in range(n):
            model.addConstr(f[k, i, j] <= C * x[k, i, j])
            model.addConstr(f[k, i, j] >= 0)
            
# 4
for k in range(v):
    model.addConstr(gp.quicksum(x[k, 0, j] for j in range(1, n)) == 1)
    model.addConstr(gp.quicksum(x[k, i, 0] for i in range(1, n)) == 1)
    
# cost
total_distance = gp.quicksum(d[i][j]*x[k, i, j] for k in range(v) for i in range(n) for j in range(n))

# optimize
model.setObjective(total_distance, GRB.MINIMIZE)
model.optimize()

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i5-6400 CPU @ 2.70GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 224 rows, 196 columns and 714 nonzeros
Model fingerprint: 0xa0f64ca6
Variable types: 98 continuous, 98 integer (98 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+04]
  Objective range  [1e+01, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 112 rows and 28 columns
Presolve time: 0.00s
Presolved: 112 rows, 168 columns, 552 nonzeros
Variable types: 84 continuous, 84 integer (84 binary)
Found heuristic solution: objective 832.0000000

Root relaxation: objective 3.875128e+02, 127 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time


In [221]:
## ROUTES ##
def get_routes(x, n, v):
    routes =[[] for _ in range(v)]
    for k in range(v):
        route = [0] # start from depot
        current_city = 0
        visited = set()
        while len(route) <= n:
            for j in range(n):
                if j != current_city and x[k, current_city, j].X > 0.5:
                    route.append(j)
                    visited.add(current_city)
                    current_city = j
                    break
            if current_city == 0:
                break
        routes[k] = route
    return routes

In [222]:
## RESULTS ##
if model.status == GRB.OPTIMAL:
    print("Βρέθηκε βέλτιστη διαδρομή:")
    routes = get_routes(x, n, v)
    for k, route in enumerate(routes):
        print("Όχημα/Διαδρομή " + str(k+1) + ": ",end='')
        complete_route = [cities[i] for i in route]
        print(' -> '.join(complete_route))
    print("Συνολική απόσταση: " + str(model.ObjVal))
else:
    print("Δεν βρέθηκε βέλτιστη διαδρομή")

Βρέθηκε βέλτιστη διαδρομή:
Όχημα/Διαδρομή 1: Ω -> Γ -> Δ -> Ω
Όχημα/Διαδρομή 2: Ω -> Β -> Ε -> Α -> ΣΤ -> Ω
Συνολική απόσταση: 497.0


Επομένως οι διαδρομές που ελαχιστοποιούν το συνολικό αριθμό χιλιομέτρων που απαιτούνται για να ικανοποιηθεί η ζήτηση, όπως προκύπτει από τα παραπάνω, είναι οι εξής:

Διαδρομή $1: Ω -> Γ -> Δ -> Ω$

Διαδρομή $2: Ω -> Β -> Ε -> Α -> ΣΤ -> Ω$
